In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go

In [2]:
np.random.seed(0)
X = np.random.uniform(-10, 10, 100)
epsilon = np.random.normal(0, 0.1, 100)
y = 3 * X + 4 + epsilon

In [3]:
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1) 

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
model_standard_SGD = LinearRegressionModel()
model_custom_SGD = LinearRegressionModel()

In [5]:
def custom_gradient_descent(model, X_tensor, y_tensor, learning_rate=0.01, num_epochs=100):
    criterion  = torch.nn.MSELoss()
    loss_values = []

    for _ in range(num_epochs):
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        loss.backward()
        loss_values.append(loss.item())

        with torch.no_grad():
            model.linear.weight -= learning_rate * model.linear.weight.grad
            model.linear.bias -= learning_rate * model.linear.bias.grad
            model.linear.weight.grad.zero_()
            model.linear.bias.grad.zero_()

    return loss_values

In [6]:
def gradient_descent(model, X_tensor, y_tensor, learning_rate=0.01, num_epochs=100):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    loss_values = []

    for _ in range(num_epochs):
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())

    return loss_values

In [7]:
loss_standard = gradient_descent(model_standard_SGD, X_tensor, y_tensor)
loss_custom = custom_gradient_descent(model_custom_SGD, X_tensor, y_tensor)

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(loss_standard)), y=loss_standard, mode='lines', name='Standard SGD'))
fig.add_trace(go.Scatter(x=np.arange(len(loss_custom)), y=loss_custom, mode='lines', name='Custom SGD'))

fig.update_layout(
    title="Training Loss",
    xaxis=dict(title="Epochs"),
    yaxis=dict(title="Loss (MSE)"),
    showlegend=True
)

fig.show(config=dict(scrollZoom=True))

In [9]:
predicted_standard = model_standard_SGD(X_tensor).detach().numpy()
predicted_custom = model_custom_SGD(X_tensor).detach().numpy()

scatter = go.Scatter(x=X, y=y, mode='markers', name='True data')
line_standard = go.Scatter(x=X, y=predicted_standard.flatten(), mode='lines', name='Fitted line standard SGD', line=dict(color='red'))
line_custom = go.Scatter(x=X, y=predicted_custom.flatten(), mode='lines', name='Fitted line custom SGD', line=dict(color='green'))

layout = go.Layout(
    title="Linear Regression Fit",
    xaxis=dict(title="X"),
    yaxis=dict(title="y"),
    showlegend=True
)

fig = go.Figure(data=[scatter, line_standard, line_custom], layout=layout)
fig.show(config=dict(scrollZoom=True))

print(f"Learned parameters (standard): W = {model_standard_SGD.linear.weight.item():.4f}, b = {model_standard_SGD.linear.bias.item():.4f}")
print(f"Learned parameters (custom): W = {model_custom_SGD.linear.weight.item():.4f}, b = {model_custom_SGD.linear.bias.item():.4f}")

Learned parameters (standard): W = 2.9884, b = 3.3434
Learned parameters (custom): W = 2.9888, b = 3.3666
